In [42]:
import mysql.connector as connector
import pandas as pd
import datetime as dt

In [43]:
# Make a connection:
try:

    connection = connector.connect(user= 'root' , password = '******' )
    print('Connected...')

except connector.Error as err:

    print('Error code: ', err.errno)
    print('Error message: ', err.msg)

Connected...


In [44]:
# Create the cursor:

if connection.is_connected():
    cursor = connection.cursor(buffered=True)
    print('cursor ready...')
else:
    print("It is not possible to create the cursor!" )

cursor ready...


In [45]:
# Set the database to work with:

cursor.execute('USE meta_py_to_mysql')
print('The database is setted to: ', connection.database)

The database is setted to:  meta_py_to_mysql


In [46]:
# See all the tables in the database:
cursor.execute('SHOW TABLES')
for i in cursor:
    print(i)

('bookings',)
('menu_items',)
('menus',)
('orders',)


In [8]:
# Deep look into bookings table:

cursor.execute('SELECT * FROM bookings')
result1 = cursor.fetchall()
columns1 = cursor.column_names

df1 = pd.DataFrame(data=result1, columns=columns1)
df1

,bookingID,tableNo,guestFirstName,guestLastName,bookingSlot,employeeID
0,1,12,Anna,Iversen,0 days 19:00:00,1
1,2,12,Joakim,Iversen,0 days 19:00:00,1
2,3,19,Vanessa,McCarthy,0 days 15:00:00,3
3,5,5,Hiroki,Yamane,0 days 18:30:00,2
4,6,10,Diana,Pinto,0 days 20:00:00,4


In [24]:
# Get all the booking slots from database and add 1 hour to then:

cursor.execute('SELECT * FROM bookings')

for row in cursor:
    booking_id = row[0]
    booking_slot = row[4]
    new_booking_slot = booking_slot + dt.timedelta(hours=1) #timedelta adds some choosed time
    print(f'booking id {booking_id} is moved from {booking_slot} to {new_booking_slot}')

booking id 1 is moved from 19:00:00 to 20:00:00
booking id 2 is moved from 19:00:00 to 20:00:00
booking id 3 is moved from 15:00:00 to 16:00:00
booking id 5 is moved from 18:30:00 to 19:30:00
booking id 6 is moved from 20:00:00 to 21:00:00


In [25]:
# Deep look into orders table:

cursor.execute('SELECT * FROM orders')
result2 = cursor.fetchall()
columns2 = cursor.column_names

df2 = pd.DataFrame(data=result2, columns=columns2)
df2

,orderID,tableNo,menuID,bookingID,billAmount,quantity
0,1,12,1,1,2,86
1,2,19,2,2,1,37
2,3,15,2,3,1,37
3,4,5,3,4,1,40
4,5,8,1,5,1,43


In [38]:
# Use of the function SUM():

cursor.execute("""SELECT SUM(billAmount) AS total_sale FROM orders """)
result3 = cursor.fetchone()
columns3 = cursor.column_names

print(columns3[0], ' = ', result3[0])

total_sale  =  6


In [68]:
# Use of the function HOUR() to extract only the hour from the bookingSlot column:

cursor.execute("""SELECT bookingID, HOUR(bookingSlot) FROM bookings""")
result4 = cursor.fetchall()
columns4 = cursor.column_names

df3 = pd.DataFrame(data=result4, columns=columns4)
df3.style.hide(axis='index')

bookingID,HOUR(bookingSlot)
1,19
2,19
3,15
5,18
6,20


In [60]:
# Use upper() and concat() to get the guests complete name in uppercase:

cursor.execute("""SELECT bookingID, UPPER(CONCAT(guestFirstName,' ' ,guestLastName)) AS name FROM bookings""")
result5 = cursor.fetchall()
columns5 = cursor.column_names

df4 = pd.DataFrame(data=result5, columns=columns5)
df4.style.hide(axis='index')

bookingID,name
1,ANNA IVERSEN
2,JOAKIM IVERSEN
3,VANESSA MCCARTHY
5,HIROKI YAMANE
6,DIANA PINTO


In [74]:
# Varios outputs in the same dataframe
"""
- Number of bookings
- Total sale
- Average sale
"""
cursor.execute("""SELECT COUNT(*) AS n_bookings FROM bookings""")
n_bookings = cursor.fetchone()

cursor.execute("""SELECT SUM(billAmount) FROM orders""")
total_sale = cursor.fetchone()

cursor.execute("""SELECT AVG(billAmount) FROM orders""")
avg_sale = cursor.fetchone()

print('Today statistics:')
print('Number of bookings: ',n_bookings[0])
print('Total sale: ', total_sale[0])
print('Average sale: ', avg_sale[0])

Today statistics:
Number of bookings:  5
Total sale:  6
Average sale:  1.2000


In [83]:
# Varios outputs in the same dataframe:
"""
- Number of bookings
- Total sale
- Average sale
"""
cursor.execute("""SELECT 
COUNT(bookingID) AS n_bookings,
SUM(billAmount) AS total_sale,
AVG(billAmount) AS avg_sale
FROM orders;""")

results = cursor.fetchall()[0]

print('Today statistics:')
print('Number of bookings: ',results[0])   
print('Total sale: ', results[1])
print('Average sale: ', results[2])

Today statistics:
Number of bookings:  5
Total sale:  6
Average sale:  1.2000


In [87]:
# Count the number of bookings for each table:

cursor.execute("""SELECT tableNo AS 'table_number',
COUNT(tableNo) AS 'n_bookings' 
FROM bookings 
GROUP BY tableNo 
ORDER BY n_bookings DESC;""")

result6 = cursor.fetchall()
columns6 = cursor.column_names

df6 = pd.DataFrame(data=result6, columns=columns6)
df6.style.hide(axis='index')

table_number,n_bookings
12,2
19,1
5,1
10,1


In [88]:
'''
Modify the table bookings replacing the booking slot time with:

* Late afternoon: for hours 15 and 16  
* Evening: for hours 17 and 18 
* Night: for hours 19 and 20 

'''

sql_query="""
SELECT
BookingID,
CONCAT(GuestFirstName,' ',GuestLastName) AS Guest_Name,

CASE
WHEN HOUR(BookingSlot) IN (15,16) THEN "Late afternoon" 
WHEN HOUR(BookingSlot) IN (17,18) THEN "Evening" 
WHEN HOUR(BookingSlot) IN (19,20) THEN "Night"
ELSE "Time not available" 
END AS Arrival_slot

FROM Bookings;"""

cursor.execute(sql_query)
result7 = cursor.fetchall()
columns7 = cursor.column_names

df7 = pd.DataFrame(data=result7,columns=columns7)
df7.style.hide(axis='index')

BookingID,Guest_Name,Arrival_slot
1,Anna Iversen,Night
2,Joakim Iversen,Night
3,Vanessa McCarthy,Late afternoon
5,Hiroki Yamane,Evening
6,Diana Pinto,Night


In [89]:
# Close the connection:

if connection.is_connected():
    cursor.close()
    print('Cursor closed...')
    connection.close()
    print('Disconnected...')
else:
    print('Already disconnected')

Cursor closed...
Disconnected...
